In [1]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from loop import TrainingLoop
from batchSelection import bad_batch_selector

In [2]:
# Loading the dataset from the files saved in the preprocessing notebook.
path = 'data/music_genres'
prefix = 'music_genres_'
X_train = np.load(os.path.join(path, prefix+'train_vectors.npy'))
y_train = np.load(os.path.join(path, prefix+'train_labels.npy'))
X_test  = np.load(os.path.join(path, prefix+'test_vectors.npy'))
y_test  = np.load(os.path.join(path, prefix+'test_labels.npy'))

In [3]:
def build_model():
    # Setting up the model.
    model = Sequential()
    model.add(Dense(250, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.4))
    model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.4))
    model.add(Dense(50, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    return model

In [4]:
def train(model, X_train, y_train, batch_selection, epochs):
    log_dir = {bad_batch_selector: 'windowed', None: 'original'}
    # Put the model in our custom training loop.
    TrainingLoop(
        model,
        X_train,
        y_train,
        validation_split = 0.1,
        batch_size = 12,
        optimizer = tf.keras.optimizers.Adam(),
        loss_function = tf.keras.losses.CategoricalCrossentropy(),
        train_metrics = tf.keras.metrics.CategoricalAccuracy(),
        val_metrics = tf.keras.metrics.CategoricalAccuracy(),
        batch_selection = batch_selection,
        log_file = os.path.join('logs', log_dir[batch_selection], 'music_genres.csv')
    ).train(epochs)  # Training model.

In [5]:
# Set random seed so the comparison of different solutions won't be affected by it.
tf.random.set_seed(42)
np.random.seed(42)

# Train model with default batch selection.
model = build_model()
train(model, X_train, y_train, batch_selection=None, epochs=50)

Epoch 1/50	Loss: 1.8535	Metrics: 0.2055: 	Validation metrics: 0.375: 	100% | 60/60 [00:07<00:00,  8.36it/s]
Epoch 2/50	Loss: 1.5358	Metrics: 0.3638: 	Validation metrics: 0.4444: 	100% | 60/60 [00:01<00:00, 39.01it/s]
Epoch 3/50	Loss: 1.4155	Metrics: 0.4583: 	Validation metrics: 0.5138: 	100% | 60/60 [00:01<00:00, 57.58it/s]
Epoch 4/50	Loss: 1.6259	Metrics: 0.4819: 	Validation metrics: 0.4722: 	100% | 60/60 [00:00<00:00, 63.29it/s]
Epoch 5/50	Loss: 1.3714	Metrics: 0.5430: 	Validation metrics: 0.5138: 	100% | 60/60 [00:00<00:00, 67.04it/s]
Epoch 6/50	Loss: 1.4894	Metrics: 0.5722: 	Validation metrics: 0.5277: 	100% | 60/60 [00:01<00:00, 58.82it/s]
Epoch 7/50	Loss: 1.5168	Metrics: 0.5638: 	Validation metrics: 0.5277: 	100% | 60/60 [00:01<00:00, 47.58it/s]
Epoch 8/50	Loss: 1.6092	Metrics: 0.6013: 	Validation metrics: 0.5416: 	100% | 60/60 [00:01<00:00, 32.28it/s]
Epoch 9/50	Loss: 1.4755	Metrics: 0.6041: 	Validation metrics: 0.5833: 	100% | 60/60 [00:01<00:00, 44.12it/s]
Epoch 10/50	Loss: 1.

In [6]:
# We still have to compile the model for the test evaluation.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# After compiling we can run the evaluation.
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 7ms/step - loss: 5.2332 - accuracy: 0.6850


[5.233212947845459, 0.6850000023841858]

In [7]:
tf.random.set_seed(42)
np.random.seed(42)

# Train model with windowed batch selection algorithm.
model = build_model()
train(model, X_train, y_train, batch_selection=bad_batch_selector, epochs=50)

Epoch 1/50	Loss: 2.0496	Metrics: 0.1958: 	Validation metrics: 0.4305: 	100% | 60/60 [00:06<00:00,  9.68it/s]
Epoch 2/50	Loss: 1.5194	Metrics: 0.2874: 	Validation metrics: 0.4166: 	100% | 60/60 [00:02<00:00, 27.62it/s]
Epoch 3/50	Loss: 1.4034	Metrics: 0.3597: 	Validation metrics: 0.4583: 	100% | 60/60 [00:02<00:00, 24.06it/s]
Epoch 4/50	Loss: 1.6548	Metrics: 0.4486: 	Validation metrics: 0.4722: 	100% | 60/60 [00:02<00:00, 29.73it/s]
Epoch 5/50	Loss: 1.3414	Metrics: 0.4430: 	Validation metrics: 0.4722: 	100% | 60/60 [00:01<00:00, 31.15it/s]
Epoch 6/50	Loss: 1.3222	Metrics: 0.4986: 	Validation metrics: 0.5555: 	100% | 60/60 [00:02<00:00, 26.93it/s]
Epoch 7/50	Loss: 1.5119	Metrics: 0.5319: 	Validation metrics: 0.5555: 	100% | 60/60 [00:03<00:00, 19.61it/s]
Epoch 8/50	Loss: 1.6802	Metrics: 0.5402: 	Validation metrics: 0.5277: 	100% | 60/60 [00:03<00:00, 18.50it/s]
Epoch 9/50	Loss: 1.2591	Metrics: 0.5680: 	Validation metrics: 0.5277: 	100% | 60/60 [00:01<00:00, 31.15it/s]
Epoch 10/50	Loss: 1

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 5.2678 - accuracy: 0.6600


[5.26779317855835, 0.6600000262260437]